# Preparing Layers
This code takes layers directly downloaded from the Data BC catalogue and prepares them for the creation of a database for landscape level planning. 
There are a few starting steps that cannot be automated as they are specific to each project.

    1. Create a feature class of the area of interest (AOI) for the dataset. 
    2. Download data - users might need to use their AOI to work around max download amounts.
Note: for those downloading from the DataBC site, and using an AOI to define the interest area they downloaded layers are not clipped to the AOI but rather are clipped to the smallest square (or rectangle that contains all points of the AOI). This means that if you have an AOI that is in different sections (i.e. a planning area that is not continuous) it will use a rectangle that covers all the points. This may require splitting the AOI into two sections. This case is not included in the code, but it can be adapted to accomidate this.

    3. Create a file geodatabase named "Data_Inputs.gdb" in a file location that makes logical sense for the analysis. 
    4. Create a file geodatabase named "Data_Prep.gdb" in a file location that makes logical sense for the analysis. 

In [2]:
# User definitions:

#arc envrironment
#Set the file directory to a .gdb that hosts all the raw input data for analysis
##Recomended that this .gdb is called Data_Inputs.gdb

Data_Inputs = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Data_Inputs.gdb"

#File location to the file directory that host the.gdb where the cleaned up data files will go. 
##Recomended to call this .gdb Data_Prep.gdb

Data_Prep = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Data_Prep.gdb"

#Define your AOI file
AOI = "GoldenBear_AOI"






In [3]:
import arcpy
import time

Start = time.time()

# Define the work environment and the datasets/feature classes to work with.
# The workspace will be the file location of the Data_Inputs.gdb and needs to be defined by the user.
# out_GDB is where the newly created and prepared layers will be created.

arcpy.env.workspace = Data_Inputs
out_GDB = Data_Prep

# Feature classes definition
# This section of code defines the differenet layers and provides them with a short name for easier referencing.
# This code is set up to use the layer names of data directly downloaded from the DataBC catalogue. 
# Make sure to change the name of the AOI layer to match what you have called this feature class. 
# If you are missing some layers, simply comment out those lines here and in the following dictionary step.

Agric = "WHSE_LEGAL_ADMIN_BOUNDARIES_OATS_ALR_POLYS"
BEC = "BEC"
Caribou = "WHSE_WILDLIFE_INVENTORY_GCPB_CARIBOU_POPULATION_SP"
IR = "WHSE_ADMIN_BOUNDARIES_CLAB_INDIAN_RESERVES"
FN_SOI = "QSOI_BCREG_polygon"
Lakes = "WHSE_LAND_AND_NATURAL_RESOURCE_EAUBC_LAKES_SP"
LU = "WHSE_LAND_USE_PLANNING_RMP_LANDSCAPE_UNIT_SVW"
OGMA = "WHSE_LAND_USE_PLANNING_RMP_OGMA_LEGAL_CURRENT_SVW"
Parks_Fed = "WHSE_ADMIN_BOUNDARIES.CLAB_NATIONAL_PARKS"
Parks_Prv = "WHSE_ADMIN_BOUNDARIES_ADM_BC_PARKS_REGIONS_SP"
Rec = "WHSE_FOREST_VEGETATION_REC_VISUAL_LANDSCAPE_INVENTORY"
Results = "WHSEFOREST_VEGETATION.VEG_CONSOLIDATED_CUT_BLOCKS_SP"
Rivers = "WHSE_BASEMAPPING_FWA_RIVERS_POLY"
F_Roads = "WHSE_FOREST_TENURE.FTEN_ROAD_LINES"
Roads = "DRA_MPAR_line"
Streams = "WHSE_BASEMAPPING_FWA_STREAM_NETWORKS_SP"
F_Own = "F_OWN_polygon"
UWR = "WHSE_WILDLIFE_MANAGEMENT_WCP_WILDLIFE_HABITAT_AREA_POLY"
VRI = "GoldenBear_R1_Poly"
Watersheds = "COM_WS_PUB_polygon"
Geo_watersheds = "WHSE_FISH_WDIC_WATERSHED_GROUP_POLY"
Wetlands = "WHSE_BASEMAPPING_FWA_WETLANDS_POLY"
WHA_a = "WHSE_WILDLIFE_MANAGEMENT_WCP_WILDLIFE_HABITAT_AREA_POLY"
WHA_p = "WHA_proposed"

# Dictionary of all feature classes; 
# dict key = variable defined above for each feature class name in the workspace GDB
# Make sure that all of the layers you defined above are included here. 
# Comment out any layers that are missing. 

layersDict = {
    aoi: "AOI",
    Agric: "Agric",
    BEC: "BEC",
    Caribou: "Caribou",
    IR: "IR",
    FN_SOI: "FN_SOI",
    Lakes: "Lakes",
    LU: "LU",
    OGMA: "OGMA",
    Parks_Fed: "Parks_Fed",
    Parks_Prv: "Parks_Prv",
    Rec: "Rec",
    Results: "Results",
    Rivers: "Rivers",
    F_Roads: "F_Roads"
    Roads: "Roads",
    Streams: "Streams",
    F_Own: "F_Own",
    UWR: "UWR",
    VRI: "VRI",
    Watersheds: "Watersheds",
    Geo_watersheds: "Geo_watersheds",
    Wetlands: "Wetlands",
    WHA_a: "WHA_a",
    WHA_p: "WHA_p"
}

# Project feature classes to a common coordinate system defined by VRI
# This section of code ensure that each layer is using the same projection and coordinate system.
# It uses the VRI as reference, and would work with any similar dataset.
# It creates a new layer for each of the defined layers called DEFINEDNAME_proj. Here _proj stands for projection.
# These layers will be created in the Data_Inputs.gdb
# There will be no layer created called VRI_proj, since we are using the VRI as the reference data. We will account for this in the following step. 

desc = arcpy.Describe(VRI)
sr = desc.spatialReference

for m in layersDict.keys():
    if layersDict[m] != "VRI":
        arcpy.Project_management(m, layersDict[m]+"_proj", sr)

print("Data projection finished. Clipping to AOI...")

# Clip all feature classes to AOI_proj
# This section clips all the layers to AOI_proj and exports them to the Data_Prep.gdb.

#This section works with all layers excluding the aoi and VRI layers
for m in layersDict.keys():
    if layersDict[m] not in [aoi, "VRI"]:
        arcpy.Clip_analysis(layersDict[m] + "_proj", "AOI_proj", out_GDB + "\\" + layersDict[m] + "_proj_clip")

#This section clips the VRI and accounts for its different name in the step above.
arcpy.Clip_analysis(VRI, "AOI_proj", out_GDB + "\\" + "VRI" + "_proj_clip")

print("Clipping to AOI_proj finished. Repairing geometry...")

# Repair Geometry
# This section fixes any geometry errors. This tool does not create a new feature class.
# Instead it repairs any self intersections, invalid records. ect. 

arcpy.env.workspace = out_GDB
for m in layersDict.keys():
     arcpy.RepairGeometry_management(layersDict[m]+"_proj_clip")

print('It took', round((time.time() - Start) / 60, 1), "minutes to prepare the layers.")







ModuleNotFoundError: No module named 'arcpy'

# Buffer Features
The next section of code creates a buffer around rivers, streams, wetlands, lakes and roads. 

If there are other layers that you would like to buffer simply add in some extra lines of code and define the layers. 

In [ ]:
## this script buffers
#rivers at 100m
#streams at 20m
#wetlands at 10m
#lakes at 30m
#roads right-of-way at 10m

Start = time.time()

#define work environment and datsets/fetaure classes to work with.
arcpy.env.workspace = Data_Prep

# Feature classes definition.
lakes="Lakes_proj_clip"
rivers="Rivers_proj_clip"
F_roads = "F_Roads_proj_clip"
roads="Roads_proj_clip"
streams="Streams_proj_clip"
wetlands="Wetlands_proj_clip"

#buffering
arcpy.Buffer_analysis(lakes, lakes+"_buff", "30 meters", "OUTSIDE_ONLY", "", "ALL", "")
print ("finished Buffering lakes. buffering rivers...")

arcpy.Buffer_analysis(rivers, rivers+"_buff", "100 meters", "OUTSIDE_ONLY", "", "ALL", "")
print ("finished Buffering rivers. buffering roads...")

arcpy.Buffer_analysis(F_roads, F_roads+"_buff", "10 meters", "FULL", "", "ALL", "")
print ("finished Buffering forest roads. buffering streams...")

arcpy.Buffer_analysis(roads, roads+"_buff", "10 meters", "FULL", "", "ALL", "")
print ("finished Buffering roads. buffering streams...")

arcpy.Buffer_analysis(streams, streams+"_buff", "20 meters", "FULL", "", "ALL", "")
print ("finished Buffering streams. buffering wetlands...")

arcpy.Buffer_analysis(wetlands, wetlands+"_buff", "10 meters", "OUTSIDE_ONLY", "", "ALL", "")
print ("finished Buffering wetlands...")

print('It took', round((time.time() - Start) / 60, 1), "minutes to buffer the features.")

After running the above section of code the user must manually fix the topology errors that the different layers may contain. 
The most common topology error is when there is an overlap between two polygons or a small gap. For landscape level planning purposes we cannot have either of these and they must be correct. 

Follow the steps provided:

    1. Right click on the Data_Prep.gdb in the catalog window.
    2. Select "New" then "Feature Dataset". 
    3. Name the new feature dataset "fds". 
    4. Import the correct coordinate system for the first feature class.
    5. Leave all other options as the default.
    6. Drag the first feature class into the new feature dataset (fds).
    7. Right click on fds and select "new" then "create new topology".
    8. Define a xy cluster tolerance rule of 0.1m
    9. Select the layer you dragged in and click next.
    10. Select the layer, and set the rules to "Must Not Overlap". 
    11. Add an additional rule for that layer of "Must Not Have Gaps".
    12. Click next.
    13. Double check options and click "Finish"
    14. Once validated, right click on the topology class created in fds. Select "properties".
    15. Navigate to "Errors" and click "Generate Summary".
    16. The report will show how many errors. 
    17. If none then the topology class created for the first feature class can be deleted.
    18. The feature class can be dragged back into the Data_Prep.gdb.
    19. The next layer can be dragged into fds.
    20. Complete steps for all feature classes

To repair overlap errors:

    1. Drag the topology feature that has errors into the Map. 
    2. In the Map window erros are topology error are shown in red. 
    3. Add the "Topology" toolbar.
    4. Right click the feature class from the table of contents. Select "Edit Features" then "Start Editing".
    5. Add the "Editor" toolbar.
    6. From the Topology toolbar open the error inspector.
    7. Search for overlap errors.
    8. Select the first error, right click, and select "Merge".
    9. Repair Gap errors, but right clicking a gap error in the "Error Inspector" window.
    10. Select "Zoom To" then "Create Feature".
    11. Select the new feature and an adjacent polygon. From the "Editor" pull-down in the "Editor" toolbar select merge. 
    12. Fix all topology erros.
    13. When all are corrected soom to the entire feature class. 
    14. In the "topology" toolbar select "Validate Topology in Current Extent".
    15. Examine the "Error Inspector" window for other errors.
    16. Repeat until all errors are repaired.
    17. In the "Editor" menu, select "Stop Editing" and "Yes" to save the edits. 
    18. You can now removed the topology and feature class layers from the Map.
    19. The topology class can be deleted and the feature class dragged into the Data_Prep.gdb.
    
It is possible to check for Topology errors in multiple layers at once (if they share a projection system), but sometimes it messes up. It is best to work one layer at a time until all errors are fixed.

With data from DataBC there should be few to no erros, but you still need to check. 

Sometimes when you move the feature class out of fds you might get the error "Move Failed. Cannot aquire a schema lock becuause of an existing lock". Just refesh fds and try again. 

# Attribute Clean Up
In the previous sets new feature classes were built that are clipped to the AOI, were buffered and had topology erros fixed. This next step removes the columns in the feature classes attribute tables that are not needed for analysis. 

The goal is to only include the fields that are required. 

In this step we will also check to see if a feature class has any features. It isn't uncommon that after collecting the data and clipping to size that there might not be any features. For example, there may not be any OGMA's that overlap with your AOI. These layers can be removed from this and future steps.

The following code uses the Delete field geoprocessing tool to remove extra fields. 
This code was developed for the data avaliable through the Data_BC Catalog
Remember that you don't need to worry about the features that were buffered (all fields were removed during the buffering step).

It is highly recommended that you check each features attribute table after this step. Some feaures have two rows named OBJECTID, one that is removable and one that is not. Delete one manually if possible.


In [ ]:
import arcpy
import time

Start = time.time()

# Define the work environment and the datasets/feature classes to work with. 
arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Data_Prep.gdb"

# Feature classes definition
aoi="AOI_proj_clip"
Agric = "Agric_proj_clip"
BEC = "BEC_proj_clip"
Caribou = "Caribou_proj_clip"
IR = "IR_proj_clip"
FN_SOI = "FN_SOI_proj_clip"
LU = "LU_proj_clip"
OGMA = "OGMA_proj_clip"
Parks_Fed = "Parks_Fed_proj_clip"
Parks_Prv = "Parks_Prv_proj_clip"
Rec = "Rec_proj_clip"
F_own = "F_Own_proj_clip"
UWR = "UWR_proj_clip"
Watersheds = "Watersheds_proj_clip"
Geo_watersheds = "Geo_watersheds_proj_clip"
WHA_a = "WHA_a_proj_clip"
WHA_p = "WHA_p_proj_clip"

# Delete fields from the Agric
print("Deleting fields from Agric...")

# Get a list of all fields in the feature class
fields = [field.name for field in arcpy.ListFields(Agric)]

# Determine the fields to keep (containing "SHAPE" and "OBJECTID")
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID" in field]

# Determine the fields to delete
fields_to_delete = [field for field in fields if field not in fields_to_keep]

# Delete the fields from the feature class
arcpy.DeleteField_management(Agric, fields_to_delete)

# Delete fields from the AOI
print("Deleting fields from aoi...")
fields = [field.name for field in arcpy.ListFields(aoi)]
fields_to_keep = [field for field in fields if "Shape" in field or "OBJECTID" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(aoi, fields_to_delete)

# Delete fields from the BEC
print("Deleting fields from BEC...")
fields = [field.name for field in arcpy.ListFields(BEC)]
fields_to_keep = [field for field in fields if "Shape" in field or "OBJECTID" in field or "NATURAL_DISTURBANCE" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(BEC, fields_to_delete)

# Delete fields from the Caribou
print("Deleting fields from Caribou...")
target = Caribou
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "HERD_STATUS" in field or "HERD_NAME" in field or "RISK_STATUS" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the F_Own
print("Deleting fields from F_Own...")
target = F_Own
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "Shape" in field or "OBJECTID_1" in field or "SCHEDULE" in field or "OWN_DESC" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the FN_SOI
print("Deleting fields from FN_SOI...")
target = FN_SOI
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "Shape" in field or "OBJECTID_1" in field or "NAME" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the Geo_Watershed
print("Deleting fields from Geo_watersheds...")
target = Geo_watersheds
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "WATERSHED_GROUP_NAME" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the IR
print("Deleting fields from IR...")
target = IR
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "ENGLISH_NAME" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the LU
print("Deleting fields from LU...")
target = LU
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "LANDSCAPE_UNIT_NAME" in field  or "BIODIVERSITY_EMPHASIS_OPTION" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the OGMA
print("Deleting fields from OGMA...")
target = OGMA
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the Parks_Prv
print("Deleting fields from Parks_Prv...")
target = Parks_Prv
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "PROTECTED_LANDS_NAME" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the Parks_Fed
print("Deleting fields from Parks_Fed...")
target = Parks_Fed
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "ENGLISH_NAME" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the Rec
print("Deleting fields from Rec...")
target = Rec
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "REC_VAC_FINAL_VALUE_CODE" in field or "REC_EVQO_CODE" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the UWR
print("Deleting fields from UWR...")
target = UWR
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "UWR_UNIT_NUMBER" in field or "UWR_NUMBER" in field  or "TIMBER_HARVEST_CODE" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the Watersheds
print("Deleting fields from Watersheds...")
target = Watersheds
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "Shape" in field or "OBJECTID_1" in field or "COMMUNITY_WS_NAME" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the WHA_a
print("Deleting fields from WHA_a...")
target = WHA_a
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "COMMON_SPECIES_NAME" in field or "TIMBER_HARVEST_CODE" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

# Delete fields from the WHA_p
print("Deleting fields from WHA_p...")
target = WHA_p
fields = [field.name for field in arcpy.ListFields(target)]
fields_to_keep = [field for field in fields if "SHAPE" in field or "OBJECTID*" in field or "COMMON_SPECIES_NAME" in field or "TIMBER_HARVEST_CODE" in field]
fields_to_delete = [field for field in fields if field not in fields_to_keep]
arcpy.DeleteField_management(target, fields_to_delete)

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

# Depletions
This step is for identifying areas that have been recently harvested but are not yet accounted for in the VRI. Sometimes the Results (Consolidated cutblocks) and the VRI has overlapping data. The first step to ensure you do not double count areas is to simplifiy the polygon shapes is to erase the polygones with valid (non-null) harvest dates in the VRI from the Results feature class. 

Then in "Editor: mode, manually edit the remaining polygons in Results. 
    
    Delete small isolated polygons.
    Disolve the Results by disturbance date and check and repair overlap topography errors.
    
    



In [ ]:
import arcpy
import time

Start = time.time()

#define work environment and datasets/feature classes to work with
arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Data_Prep.gdb"

Results = "Results_proj_clip"
VRI = "VRI_proj_clip"

#add fields to the Results_Opening
try:
    arcpy.AddField_management(Results, "Dist_Year", "LONG")
except:
    pass

#Update Dist_Year
with arcpy.da.UpdateCursor(Results, ["DISTURBANCE_END_DATE", "Dist_Year", "OPENING_WHEN_CREATED"]) as cursor:
    for row in cursor:
        row[1] = None
        if row[0] is not None:
            row[1] = int(str(row[0])[:4])
        else:
            row[1] = int(str(row[2])[:4])
        cursor.updateRow(row)

print('Finished updating Dist_Year field. Erase VRI harvested polygons...')

#Erases VRI
arcpy.MakeFeatureLayer_management(VRI, "VRI_harv", '"HARVEST_DATE" is not null')
arcpy.Erase_analysis(Results, "VRI_harv", "Results_erase")
print('Finished erasing VRI harvested polygons. Dissolving by Dist_Year...')

# dissolve
arcpy.Dissolve_management("Results_erase", "Results_dissolved", ["Dist_Year"], "", "SINGLE_PART", "")

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

A final visual check should be done.
Add a basemap. 
Visualize the VRI - Right click Properties, then Definition Query. Add in the Definition Query field "HARVEST_DATE is not null".
Add Results to the map viewer.
Format both layers so that they visualize as hollow with visible boundaries. 
At this point it should be possible to see if some harvested areas are missing information (aka. not listed with being harvested). They will appear as cutblocks on the basemap but will not have an outline from either the VRI or Results file. 
Any missing areas will need to have digitized as new polygons.
This is done in editor mode (add the new polygon to the Results layer). 
You will need to make an educated guess when you think the disturbance occured. This can be done by comparing it to similar looking cutblocks in the area. 

# Build Resultant File
Builds the resultant files
This section of code takes the cleaned up layers and attirbues tables and pushes them together into one layer called the resultant. 
It then tidies up the resulant file by reparing the geometry, changing multipart to single part polygons and eliminating sliver and low area polygons by merging them with their largest neighbour. 
This code builds on and updates original code y Dr. Cosmin Mann.

In [ ]:
import arcpy
import time
Start = time.time()

#eliminator function to eliminate the sliver polygons 
def eliminator(fc,fcout):
    arcpy.MakeFeatureLayer_management(fc, "tempLayer")
    arcpy.SelectLayerByAttribute_management("tempLayer", "NEW_SELECTION", '"Shape_Area" < 1000')
    arcpy.Eliminate_management("tempLayer", fcout)
    arcpy.Delete_management("tempLayer")

arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Data_Prep.gdb"

arcpy.Identity_analysis ("AOI_proj_clip", "IR_proj_clip", "temp1","ALL", 0.1)
arcpy.Identity_analysis ("temp1", "Agric_proj_clip", "temp2","ALL", 0.1)
arcpy.Identity_analysis ("temp2", "BEC_proj_clip", "temp3","ALL", 0.1)

arcpy.Identity_analysis ("temp3", "Caribou_proj_clip", "temp4","ALL", 0.1)
arcpy.Identity_analysis ("temp4", "F_Own_proj_clip", "temp5","ALL", 0.1)
arcpy.Identity_analysis ("temp5", "FN_SOI_proj_clip", "temp6","ALL", 0.1)

arcpy.Identity_analysis ("temp6", "Geo_watersheds_proj_clip", "temp7","ALL", 0.1)
arcpy.Identity_analysis ("temp7", "LU_proj_clip", "temp8","ALL", 0.1)
arcpy.Identity_analysis ("temp8", "OGMA_proj_clip", "temp9","ALL", 0.1)

arcpy.Identity_analysis ("temp9", "Parks_Fed_proj_clip", "temp10","ALL", 0.1)
arcpy.Identity_analysis ("temp10", "Parks_Prv_proj_clip", "temp11","ALL", 0.1)
arcpy.Identity_analysis ("temp11", "Rec_proj_clip", "temp12","ALL", 0.1)

arcpy.Identity_analysis ("temp12", "Results", "temp13","ALL", 0.1)
arcpy.Identity_analysis ("temp13", "UWR_proj_clip", "temp14","ALL", 0.1)
arcpy.Identity_analysis ("temp14", "Watersheds_proj_clip", "temp15","ALL", 0.1)

arcpy.Identity_analysis ("temp15", "WHA_a_proj_clip", "temp16","ALL", 0.1)
arcpy.Identity_analysis ("temp16", "WHA_p_proj_clip", "temp17","ALL", 0.1)
arcpy.Identity_analysis ("temp17", "Lakes_proj_clip", "temp18","ALL", 0.1)

arcpy.Identity_analysis ("temp18", "Rivers_proj_clip", "temp19","ALL", 0.1)
arcpy.Identity_analysis ("temp19", "wetlands_proj_clip", "temp20","ALL", 0.1)

eliminator("temp20","temp20e")
print("administrative_management completed: identifying inventory")

arcpy.Identity_analysis ("temp20e", "VRI_proj_clip", "temp21","ALL", 0.1)
arcpy.Identity_analysis ("temp21", "Lakes_proj_clip_buff", "temp22","ALL", 0.1)
arcpy.Identity_analysis ("temp22", "Rivers_proj_clip_buff", "temp23","ALL", 0.1)
arcpy.Identity_analysis ("temp23", "Roads_proj_clip_buff", "temp24","ALL", 0.1)
arcpy.Identity_analysis ("temp24", "F_Roads_proj_clip_buff", "temp25","ALL", 0.1)
arcpy.Identity_analysis ("temp25", "Streams_proj_clip_buff", "temp26","ALL", 0.1)
arcpy.Identity_analysis ("temp26", "Wetlands_proj_clip_buff", "temp27","ALL", 0.1)

print ("temp27 is the final feature dataset")
print ("repairing Geom, multi to sgl, eliminate more polys...")

arcpy.RepairGeometry_management ("temp27")
arcpy.MultipartToSinglepart_management("temp27","temp27_sgl")
eliminator("temp27_sgl","temp27_sgl_e1")
eliminator("temp27_sgl_e1","temp27_sgl_e2")
print ('It took ', round((time.time()-Start)/60,1), " minutes to run this script.")

## Manually double check the file, you may need to run the elimate tool again if there are still tiny polygons.
## The final manual steps are to delete the Temporary files that were created and to delete the FID_Temp... attributes that were created. These columns are not needed and add usless data. 
## Finally rename the data set and put into the resultant geodatabase. 

# Netdown Creation
This section of code delinates different areas of the land base. It creates three different column and sorts the data based on different values in the attribute tables.

In [ ]:
import arcpy
import time
Start = time.time()

arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Resultant_v1.gdb"

fc = "Resultant_v1" 
try:
    arcpy.AddField_management(fc, "contclass", "TEXT", "", "", 1) 
except:
    pass 
try:
    arcpy.AddField_management(fc, "rollup", "TEXT", "", "", 50) 
except:
    pass 
try:
    arcpy.AddField_management(fc, "netdown", "TEXT", "", "", 50) 
except:
    pass 
try:
    arcpy.AddField_management(fc, "THLB_Area", "Double") 
except:
    pass

#dictionary that enables the use of fields name in the update cursor 
flist = arcpy.ListFields(fc)
fdic = {} 
fl = []
for f in flist:
    fdic[f.name] = flist.index(f) 
    fl.append(f.name)

with arcpy.da.UpdateCursor(fc, fl) as cursor:
    for row in cursor:
        row[fdic["contclass"]] = "C" 
        row[fdic["netdown"]] = "THLB" 
        row[fdic["rollup"]] = "THLB"
        row[fdic["THLB_Area"]] = row[fdic["Shape_Area"]]/10000

       ## Riparian from lake, wetland, river, and Stream buffers 
        if row[fdic["FID_Lakes_proj_clip_buff"]] > 0:
            row[fdic["contclass"]] = "N" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "6_04_Lake_Buffer" 
            row[fdic["rollup"]] = "6_Riparian"
            
        elif row[fdic["FID_Wetlands_proj_clip_buff"]] > 0:
            row[fdic["contclass"]] = "N"
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = "6_03_Wetland_Buffer"
            row[fdic["rollup"]] = "6_Riparian"

        elif row[fdic["FID_Rivers_proj_clip_buff"]] > 0:
            row[fdic["contclass"]] = "N"
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = "6_02_River_Buffer"
            row[fdic["rollup"]] = "6_Riparian"

        elif row[fdic["FID_Streams_proj_clip_buff"]] > 0:
            row[fdic["contclass"]] = "N"
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = "6_01_Stream_Buffer"
            row[fdic["rollup"]] = "6_Riparian"
            
        ## Non-productive/non-commercial stands
        elif row[fdic["SPECIES_CD_1"]] not in [None, ""]:
            if row[fdic["NON_FOREST_DESCRIPTOR"]] not in [None, ""]:
                row[fdic["contclass"]] = "N"
                row[fdic["THLB_Area"]] = 0
                row[fdic["netdown"]] = ("5_03_Non_Prod_" + str(row[fdic["NON_FOREST_DESCRIPTOR"]]))[0:50]
                row[fdic["rollup"]] = "5_Non_Productive"        
     
        ##Environmentally Sensitive Areas (ESA)
        elif row[fdic["MODIFYING_PROCESS"]] not in [None, "", "N"]:
            row[fdic["contclass"]] = "N"
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = ("5_01_Sensitive_" + str(row[fdic["MODIFYING_PROCESS"]]))[0:50]
            row[fdic["rollup"]] = "5_Non_Productive"

        elif row[fdic["HARVEST_DATE"]] is None and row[fdic["SITE_INDEX"]] is not None and row[fdic["SITE_INDEX"]] < 8:
        ## and row[fdic["FID_Results"]]<1 
            row[fdic["contclass"]] = "N"
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "4_01_Poor_sites" 
            row[fdic["rollup"]] = "4_Non_Commercial"
            
        ## Reserves+Habitat
        # current data set doesn't have no harvest all is conditional harvest
        #if row[fdic["FID_UWR_proj_clip"]] >0 and row[fdic["TIMBER_HARVEST_CODE"]]=="NO HARVEST ZONE":
         #   row[fdic["contclass"]] = "N"
          #  row[fdic["THLB_Area"]] = 0
           # row[fdic["netdown"]] = ("3_04_HabitatUWR_"+ str(row[fdic["UWR_NUMBER"]]))[0:50] 
            #row[fdic["rollup"]] = "3_Reserves"

        elif row[fdic["FID_WHA_a_proj_clip"]] is not None and row[fdic["TIMBER_HARVEST_CODE_1"]]=="NO HARVEST ZONE": 
            row[fdic["contclass"]] = "N"
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = ("3_03_HabitatWHa_"+ str(row[fdic["COMMON_SPECIES_NAME"]]))[0:50] 
            row[fdic["rollup"]] = "3_Reserves"

        #if row[fdic["FID_WHA_p_proj_clip"]] >0 and row[fdic["TIMBER_HARVEST_CODE_12"]]=="NO HARVEST ZONE": 
        #    row[fdic["contclass"]] = "N"
        #    row[fdic["THLB_Area"]] = 0
        #    row[fdic["netdown"]] = ("3_03_HabitatWHp_"+ str(row[fdic["COMMON_SPECIES_NAME_1"]]))[0:50] 
        #    row[fdic["rollup"]] = "3_Reserves"

        #if row[fdic["FID_OGMA_proj_clip"]] >0: 
        #    row[fdic["contclass"]] = "N" 
        #    row[fdic["THLB_Area"]] = 0 
        #    row[fdic["netdown"]] = ("3_02_OGMA_")[0:50] 
        #    row[fdic["rollup"]] = "3_Reserves"
        
        ##Non Forest 
        #VRI Non_Forest

        elif row[fdic["BCLCS_LEVEL_1"]] in [None, "", "U"]: 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_08_Not_Typed" 
            row[fdic["rollup"]] = "2_Non_Forest"

        elif row[fdic["BCLCS_LEVEL_1"]] == "N" and row[fdic["BCLCS_LEVEL_2"]] == "L" and (row[fdic["HARVEST_DATE"]] is None or row[fdic["HARVEST_DATE"]] < 1):
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["rollup"]] = "2_Non_Forest"
            if row[fdic["BCLCS_LEVEL_3"]] in ["A", "U"]:
                row[fdic["netdown"]] = "2_07_Non_vegetated_land" 
            else:
                row[fdic["netdown"]] = "2_02_Wetlands"

        elif row[fdic["BCLCS_LEVEL_1"]] =="N" and row[fdic["BCLCS_LEVEL_2"]] =="W": 
            row[fdic["contclass"]] = "X"
            row[fdic["THLB_Area"]] = 0 
            row[fdic["rollup"]] = "2_Non_Forest"
            if row[fdic["BCLCS_LEVEL_3"]] =="A" and row[fdic["BCLCS_LEVEL_5"]] =="LA": 
                row[fdic["netdown"]] = "2_01_Lakes"
            else:
                row[fdic["netdown"]] = "2_03_Rivers"
            if row[fdic["BCLCS_LEVEL_3"]] =="W" and row[fdic["BCLCS_LEVEL_5"]] =="OC": 
                row[fdic["netdown"]] = "2_06_Ocean"
            else:
                row[fdic["netdown"]] = "2_02_Wetlands"
        
        if row[fdic["BCLCS_LEVEL_1"]] == "V" and row[fdic["BCLCS_LEVEL_2"]] == "N" and (row[fdic["HARVEST_DATE"]] is None or row[fdic["HARVEST_DATE"]] < 1):
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["rollup"]] = "2_Non_Forest"
            if row[fdic["BCLCS_LEVEL_3"]] in ["A", "U"]: 
                row[fdic["netdown"]] = "2_05_Vegetated_Not_Treed"
            else:
                row[fdic["netdown"]] = "2_02_Wetlands"
        
        elif row[fdic["BCLCS_LEVEL_1"]] == "V" and row[fdic["BCLCS_LEVEL_2"]] == "T": 
            if row[fdic["BCLCS_LEVEL_3"]] == "W":
                row[fdic["contclass"]] = "X" 
                row[fdic["THLB_Area"]] = 0 
                row[fdic["netdown"]] = "2_02_Wetlands" 
                row[fdic["rollup"]] = "2_Non_Forest"

        elif row[fdic["BCLCS_LEVEL_3"]] == "U" and row[fdic["SPECIES_CD_1"]] in [None, ""] and (row[fdic["HARVEST_DATE"]] is None or row[fdic["HARVEST_DATE"]] < 1):
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_08_Not_Typed" 
            row[fdic["rollup"]] = "2_Non_Forest"
            
        #Roads
        elif row[fdic["FID_Roads_proj_clip_buff"]] >0 : 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_04_Roads" 
            row[fdic["rollup"]] = "2_Non_Forest"
            
        #if row[fdic["FID_F_Roads_clip_buff"]] >0 : 
            #row[fdic["contclass"]] = "X" 
            #row[fdic["THLB_Area"]] = 0 
            #row[fdic["netdown"]] = "2_04_Roads" 
            #row[fdic["rollup"]] = "2_Non_Forest"
            
        #Water
        elif row[fdic["FID_Wetlands_proj_clip_buff"]]>0 : 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_02_Wetlands" 
            row[fdic["rollup"]] = "2_Non_Forest"

        elif row[fdic["FID_Rivers_proj_clip"]]>0 : 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "2_03_Rivers" 
            row[fdic["rollup"]] = "2_Non_Forest"

        elif row[fdic["FID_Lakes_proj_clip"]]>0 : 
            row[fdic["contclass"]] = "X"
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = "2_01_Lakes" 
            row[fdic["rollup"]] = "2_Non_Forest"
            
         ## Exclusions
        elif row[fdic["FID_Parks_Prv_proj_clip"]] >0:
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = ("1_03_Park_"+ str(row[fdic["REGION_NAME"]]))[0:50] 
            row[fdic["rollup"]] = "1_Exclusions"

        #if row[fdic["FID_Parks_Fed_proj_clip"]] >0:
        #    row[fdic["contclass"]] = "X" 
        #    row[fdic["THLB_Area"]] = 0
        #    row[fdic["netdown"]] = ("1_03_Park_"+ str(row[fdic["ENGLISH_NAME_1"]]))[0:50] 
        #    row[fdic["rollup"]] = "1_Exclusions"
    
        elif row[fdic["FID_IR_proj_clip"]] is not None and row[fdic["FID_IR_proj_clip"]] > 0: 
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0 
            row[fdic["netdown"]] = "1_02_AgricRsv" 
            row[fdic["rollup"]] = "1_Exclusions"
    
        elif row[fdic["FID_IR_proj_clip"]]  is not None and row[fdic["FID_IR_proj_clip"]] > 0:
            row[fdic["contclass"]] = "X" 
            row[fdic["THLB_Area"]] = 0
            row[fdic["netdown"]] = ("1_01_IR_"+ str(row[fdic["ENGLISH_NAME"]]))[0:50] 
            row[fdic["rollup"]] = "1_Exclusions"

        cursor.updateRow(row)
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

# Add starting age column
This script adds fields and updates the stand age to year 2023. 

In [ ]:
import arcview
import arcpy
import time
Start = time.time()

arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Resultant_v1.gdb"

fc = "Resultant_v1"

try:
    arcpy.AddField_management(fc, "Age_2023", "LONG")
except:
    pass

## short code to enable the use of field names
flist = arcpy.ListFields(fc)
fdic = {}
fl = []
for f in flist:
    fdic[f.name] = flist.index(f)
    fl.append(f.name)
    
#the update cursor
with arcpy.da.UpdateCursor(fc, fl) as cursor:
    for row in cursor:
        row[fdic["Age_2023"]]=None
        #if row[fdic["FID_Results"]]>0:
            #row[fdic["Age_2023"]]=2015-row[fdic["Dist_Year"]]
        if row[fdic["PROJ_AGE_1"]] != None:
            row[fdic["Age_2023"]] = row[fdic["PROJ_AGE_1"]]+2
        elif row[fdic["HARVEST_DATE"]]!= None:
            row[fdic["Age_2023"]]=2015-int(str(row[fdic["HARVEST_DATE"]])[:4])
        cursor.updateRow(row)
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

# Add AU fields
Adds AU filds and updates the AU based on the AU definitions from the Cassiar TSA.
The coastal/transitional groupings are the CWH, ESSF,
ICH, SBS, and MH zones, while the interior groupings are the BWBS and SWB zones.

Leading species present in area: AT, BL, HM, PL, SX

In [ ]:
import arcpy
import time

Start = time.time()

arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Resultant_v1.gdb"

fc = "Resultant_v1"

try:
    arcpy.AddField_management(fc, "AU", "LONG")
except arcpy.ExecuteError:
    pass

# Short code to enable the use of field names
flist = arcpy.ListFields(fc)
fdic = {f.name: flist.index(f) for f in flist}
fl = list(fdic.keys())

# Define species lists
Aspen = ['AC', 'ACT', 'AT', 'EP', 'VB', 'MB']
Bal = ['B', 'BA', 'BG', 'BL']
Cedar = ['CW', 'YC']
Alder = ['D', 'DR']
DougFir = ['F', 'FD', 'FDC', 'FDI']
Hem = ['H', 'HM', 'HW']
Pine = ['PA', 'PL', 'PLC', 'PW', 'PLI', 'PY']
Spruce = ['S', 'SS', 'SW', 'SX', 'SE', 'SXW', 'SB']

# The update cursor
with arcpy.da.UpdateCursor(fc, fl) as cursor:
    for row in cursor:
        
        # Stands
        species_cd_1 = row[fdic["SPECIES_CD_1"]]
        site_index = row[fdic["SITE_INDEX"]]
        bec_zone_code = row[fdic["BEC_ZONE_CODE"]]
        herd_name = row[fdic["HERD_NAME"]] 

        # Put Decid into its own group
        if species_cd_1 in Aspen:
            row[fdic["AU"]] = 33

        # Coastal-Spruce
        elif species_cd_1 in Spruce and bec_zone_code in ["CWH", "ESSF", "ICH", "SBS", "MH"]:
            if site_index is None or 17.1 <= site_index <= 25.0:
                row[fdic["AU"]] = 2
            elif site_index >= 25.1:
                row[fdic["AU"]] = 1
            elif 0.1 < site_index < 17.1:
                row[fdic["AU"]] = 3

        # Interior-Spruce
        elif species_cd_1 in Spruce and bec_zone_code in ["BWBS", "SWB"]:
            if site_index is None or 17.1 <= site_index <= 25.0:
                row[fdic["AU"]] = 5
            elif site_index >= 25.1:
                row[fdic["AU"]] = 4
            elif 0.1 < site_index < 17.1:
                row[fdic["AU"]] = 6

        # Alpine Spruce
        elif species_cd_1 in Spruce and bec_zone_code in ["BAFA", "CMA"]:
            row[fdic["AU"]] = 7

        # Coastal Pine
        elif species_cd_1 in Pine and bec_zone_code in ["CWH", "ESSF", "ICH", "SBS", "MH"]:
            if site_index is None or 14.6 <= site_index <= 19.0:
                row[fdic["AU"]] = 9
            elif site_index >= 19.1:
                row[fdic["AU"]] = 8
            elif 0.1 < site_index < 14.6:
                row[fdic["AU"]] = 10

        # Interior Pine
        elif species_cd_1 in Pine and bec_zone_code in ["BWBS", "SWB"]:
            if site_index is None or 14.6 <= site_index <= 19.0:
                row[fdic["AU"]] = 12
            elif site_index >= 19.1:
                row[fdic["AU"]] = 11
            elif 0.1 < site_index < 14.6:
                row[fdic["AU"]] = 13

        # Alpine Pine
        elif species_cd_1 in Pine and bec_zone_code in ["BAFA", "CMA"]:
            if site_index is None or 14.6 <= site_index <= 19.0:
                row[fdic["AU"]] = 15
            elif site_index >= 19.1:
                row[fdic["AU"]] = 14
            elif 0.1 < site_index < 14.6:
                row[fdic["AU"]] = 16

   # Coastal Balsam
        if species_cd_1 in Bal and bec_zone_code in ["CWH", "ESSF", "ICH", "SBS", "MH"]:
            if site_index is None or 13.1 <= site_index <= 16.0:
                row[fdic["AU"]] = 18
            elif site_index >= 16.1:
                row[fdic["AU"]] = 17
            elif 0.1 < site_index < 13.1:
                row[fdic["AU"]] = 19

         # Interior Balsam
        elif species_cd_1 in Bal and bec_zone_code in ["BWBS", "SWB"]:
            if site_index is None or 13.1 <= site_index <= 16.0:
                row[fdic["AU"]] = 21
            elif site_index >= 16.1:
                row[fdic["AU"]] = 20
            elif 0.1 < site_index < 13.1:
                row[fdic["AU"]] = 22
                
        # Alpine Balsam
        elif species_cd_1 in Bal and bec_zone_code in ["BAFA", "CMA"]:
            row[fdic["AU"]] = 23

   # Coastal Hemlock
        if species_cd_1 in Hem and bec_zone_code in ["CWH", "ESSF", "ICH", "SBS", "MH"]:
            if site_index is None or 14.1 <= site_index <= 17.0:
                row[fdic["AU"]] = 25
            elif site_index >= 17.1:
                row[fdic["AU"]] = 24
            elif 0.1 < site_index < 14.1:
                row[fdic["AU"]] = 26

         # Interior Hemlock
        elif species_cd_1 in Hem and bec_zone_code in ["BWBS", "SWB"]:
            if site_index is None or 14.1 <= site_index <= 17.0:
                row[fdic["AU"]] = 28
            elif site_index >= 17.1:
                row[fdic["AU"]] = 27
            elif 0.1 < site_index < 14.1:
                row[fdic["AU"]] = 29
                
        # Alpine Hemlock
        elif species_cd_1 in Hem and bec_zone_code in ["BAFA", "CMA"]:
            if site_index is None or 14.1 <= site_index <= 17.0:
                row[fdic["AU"]] = 30
            elif 0.1 < site_index < 14.1:
                row[fdic["AU"]] = 31

#Caribou stands (overwrites the previous stands if these stands are in the Caribou zone)
        if herd_name in ["Level-Kawdy"]:
            row[fdic["AU"]]=32

        cursor.updateRow(row)

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

 
                
# #Caribou stands (overwrites the previous stands if these stands are in the Caribou zone)
#         if row[fdic["HERD_NAME"]]== "Itcha-Ilgachuz":
#             if row[fdic["FID_Results"]]>0:
#                 row[fdic["AU"]]=7
#             else:
#                 row[fdic["AU"]]=6

# #MDWR stands (overwrites the previous non-MPB and Caribou stands if in MDWR zone)
#         if row[fdic["FID_UWR_proj_clip"]]>1:
#             if row[fdic["SPECIES_CD_1"]] in DougFir and row[fdic["SPECIES_PCT_1"]] >=40:
#                 if row[fdic["BEC_ZONE_CODE"]] in ["IDF", "SBPS"]:
#                     if row[fdic["HARVEST_DATE"]] not in ['',None] or row[fdic["OPENING_IND"]]== "Y" or row[fdic["FID_Results"]]>0:
#                         row[fdic["AU"]]=2
#                     else:
#                         row[fdic["AU"]]=3
#                 if row[fdic["BEC_ZONE_CODE"]] in ["SBS", "ICH"]:
#                     row[fdic["AU"]]=4
#                 else:
#                     row[fdic["AU"]]=1
#             else:
#                 row[fdic["AU"]]=5

# #recent harvest blocks with no species info assumed PL_med
# if row[fdic["AU"]]==None and (row[fdic["HARVEST_DATE"]] not in ['',None] or row[fdic["FID_Results"]]>0):
# row[fdic["AU"]]=17


# Resultant Create Unique ID
This section of code add a field and populates it with a unique integer and functions to make sure that each block has a unique ID.

In [ ]:
import arcpy
import time
import tempfile
import os

Start = time.time()

# Defining the workspace
arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Resultant_v1.gdb"
fc = "Resultant_v1"

# Adding the field
try:
    arcpy.AddField_management(fc, "Block_ID", "LONG")
except:
    pass

# Create the function for auto-increment
def get_last_incremented_value():
    try:
        with open(get_increment_file_path(), "r") as file:
            return int(file.readline().strip())
    except FileNotFoundError:
        return 0

def update_last_incremented_value(value):
    with open(get_increment_file_path(), "w") as file:
        file.write(str(value))

def get_increment_file_path():
    return os.path.join(tempfile.gettempdir(), "last_incremented_value.txt")

# Get the last incremented value
rec = get_last_incremented_value()

# Create update cursor for feature class
with arcpy.da.UpdateCursor(fc, ["Block_ID"]) as cursor:
    for row in cursor:
        rec += 1
        row[0] = rec
        cursor.updateRow(row)

# Update the last incremented value
update_last_incremented_value(rec)

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

# Sum the NetDown to put into table
## Rollup

In [ ]:
import csv
import arcpy
import time

Start = time.time()

# Set the workspace location:
arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Resultant_v1.gdb"

# Identify the feature class (table to work on)
fc = "Resultant_v1"

# This bit of code enables you to reference the field name rather than referencing the row in the list by the index position (i.e. row[0], row[1])
flist = arcpy.ListFields(fc)
fdic = {}
fl = []
for f in flist:
    fdic[f.name] = flist.index(f)
    fl.append(f.name)

# This declares the dictionary that will be used to summarize the data
sumdict = {}

# Using the data access search cursor, go through all the rows. If the dictionary has a new key, add it to the list and add the area. Otherwise, if one is already added, just keep adding to the key
with arcpy.da.SearchCursor(fc, fl) as cursor:
    for row in cursor:
        if row[fdic["rollup"]] in sumdict:
            sumdict[row[fdic["rollup"]]] += row[fdic["Shape_Area"]] / 10000
        else:
            sumdict[row[fdic["rollup"]]] = row[fdic["Shape_Area"]] / 10000

# This section makes and sorts a list of the items in the sumdict dictionary and appends them into a CSV-like table
l = list(sumdict.items())
l.sort()
d = [["rollup", "Hectares"]]
for r in l:
    d.append(r)


# Specifies the output directory and filename of the CSV file
output_directory = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\01_Resultant_Stats"
output_filename = "SUM_rollup.csv"
output_path = os.path.join(output_directory, output_filename)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Open the CSV file for writing
with open(output_path, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(d)

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")



## Netdown

In [ ]:
## summarize net down

import csv
import arcpy
import time

Start = time.time()

# Set the workspace location:
arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Resultant_v1.gdb"

# Identify the feature class (table to work on)
fc = "Resultant_v1"

# This bit of code enables you to reference the field name rather than referencing the row in the list by the index position (i.e. row[0], row[1])
flist = arcpy.ListFields(fc)
fdic = {}
fl = []
for f in flist:
    fdic[f.name] = flist.index(f)
    fl.append(f.name)

# This declares the dictionary that will be used to summarize the data
sumdict = {}

# Using the data access search cursor, go through all the rows. If the dictionary has a new key, add it to the list and add the area. Otherwise, if one is already added, just keep adding to the key
with arcpy.da.SearchCursor(fc, fl) as cursor:
    for row in cursor:
        if row[fdic["netdown"]] in sumdict:
            sumdict[row[fdic["netdown"]]]=sumdict[row[fdic["netdown"]]]+row[fdic["Shape_Area"]]/10000
        else:
            sumdict[row[fdic["netdown"]]] = row[fdic["Shape_Area"]] / 10000

##This section makes and sorts a list of the items in the sumdict dictionary and appends them into a CSV like table
l = sumdict.items()
l = sorted(l)
d = [["netdown", "Hectares"]]
for r in l:
    d.append(r)

            
# Specifies the output directory and filename of the CSV file
output_directory = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\01_Resultant_Stats"
output_filename = "SUM_netdown.csv"
output_path = os.path.join(output_directory, output_filename)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Open the CSV file for writing
with open(output_path, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(d)

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")


## Analysis Units
Summarizes the AU's 

In [ ]:
import csv
import arcpy
import time
import os

Start = time.time()

# Set the workspace location:
arcpy.env.workspace = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\Resultant_v1.gdb"

# Identify the feature class (table to work on)
fc = "Resultant_v1"

# This bit of code enables you to reference the field name rather than referencing the row in the list by the index position (i.e. row[0], row[1])
flist = arcpy.ListFields(fc)
fdic = {}
fl = []
for f in flist:
    fdic[f.name] = flist.index(f)
    fl.append(f.name)

# This declares the dictionary that will be used to summarize the data
sumdict = {}

# Using the data access search cursor, go through all the rows; if the dictionary has a new key, add it to the list and add the area; otherwise, if one is already added, just keep adding to the key
with arcpy.da.SearchCursor(fc, fl) as cursor:
    for row in cursor:
        shape_area = row[fdic["Shape_Area"]]
        au_value = row[fdic["AU"]]
        if shape_area is not None and au_value is not None:  # Filter out records with None values
            if au_value in sumdict:
                sumdict[au_value] += shape_area / 10000
            else:
                sumdict[au_value] = shape_area / 10000

# This section makes and sorts a list of the items in the sumdict dictionary and appends them into a CSV-like table
l = sorted(sumdict.items())
header = [["AU", "Hectares"]]
data = header + l

# Specifies the output directory and filename of the CSV file
output_directory = r"E:\Newmont Carbon Project\Data\Data BC\00_Script_creation_Test\01_Resultant_Stats"
output_filename = "SUM_analysisunits.csv"
output_path = os.path.join(output_directory, output_filename)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Open the CSV file for writing
with open(output_path, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(data)

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")
